In [2]:
# Run only if you have errors loading the topquartile module

from pathlib import Path
import sys

root = Path().resolve().parent.parent.parent.parent
sys.path.append(root)

In [3]:
from topquartile.modules.datamodule.dataloader import DataLoader
from topquartile.modules.datamodule.transforms.covariate import (TechnicalCovariateTransform, FundamentalCovariateTransform)
from topquartile.modules.datamodule.transforms.label import BinaryLabelTransform
from topquartile.modules.datamodule.partitions import PurgedTimeSeriesPartition

In [4]:
covtrans_config = [((TechnicalCovariateTransform, dict(sma = [20, 30],
                                                       ema = [20, 30],
                                                       momentum_change=True,
                                                       volatility = [20, 30],)))]

labeltrans_config = [(BinaryLabelTransform, dict(label_duration=20,
                                                quantile=0.75))]

partition_config = dict(n_splits=5, gap=20, max_train_size=504, test_size=60, verbose=False)

In [5]:
dataloader = DataLoader(data_id='dec2024', covariate_transform=covtrans_config,
                  label_transform=labeltrans_config, partition_class=PurgedTimeSeriesPartition,
                  partition_params=partition_config)

In [6]:
folds = dataloader.get_cv_folds()

Data not yet processed. Processing now...
Reading data from: /Users/gregruyoga/gmoneycodes/topquartile/topquartile/data/dec2024.csv
Found 342 raw ticker names.
 Applying TechnicalCovariateTransform with params {'sma': [20, 30], 'ema': [20, 30], 'momentum_change': True, 'volatility': [20, 30]}
 Applying BinaryLabelTransform with params {'label_duration': 20, 'quantile': 0.75} (globally)


/Users/gregruyoga/gmoneycodes/topquartile/topquartile/modules/datamodule/transforms/label.py:171: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_copy.groupby(level=self.ticker_level_name, group_keys=False)[self.price_column]


Imputing/dropping columns based on missingness...
 Columns with missingness > threshold (41159): ['EQY_DVD_YLD_IND', 'PX_HIGH', 'PX_LOW', 'RETURN_COM_EQY', 'PX_TO_BOOK_RATIO', 'NEWS_SENTIMENT_DAILY_AVG', 'VWAP_VOLUME', 'EQY_WEIGHTED_AVG_PX', 'CUR_MKT_CAP', 'PE_RATIO', 'PX_TO_SALES_RATIO', 'OPERATING_ROIC', 'SALES_GROWTH', 'EPS_GROWTH', 'RSI_30D', 'RSI_3D', 'DEBT_TO_MKT_CAP', 'EARN_YLD', 'TURNOVER', 'TRAIL_12M_NET_INC_GROWTH', 'sma_20', 'sma_30', 'ema_20', 'ema_30', 'volatility_20', 'volatility_30', 'roc_126', 'momentum_change', '20d_stock_return', 'INDEX_RETURN', 'EXCESS_RETURN', 'label']
 Explicitly dropping columns: ['NEWS_SENTIMENT_DAILY_AVG']
 Final columns to drop: ['TURNOVER', '20d_stock_return', 'INDEX_RETURN', 'PX_HIGH', 'TRAIL_12M_NET_INC_GROWTH', 'EARN_YLD', 'label', 'RETURN_COM_EQY', 'CUR_MKT_CAP', 'RSI_3D', 'EQY_DVD_YLD_IND', 'VWAP_VOLUME', 'roc_126', 'ema_30', 'RSI_30D', 'sma_20', 'EPS_GROWTH', 'EXCESS_RETURN', 'PX_TO_SALES_RATIO', 'ema_20', 'PX_TO_BOOK_RATIO', 'SALES_GROW

/Users/gregruyoga/gmoneycodes/topquartile/topquartile/modules/datamodule/dataloader.py:339: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data_grouped_by_ticker = self.data.groupby("ticker")


In [10]:
folds[0][1].columns

Index(['NEWS_HEAT_PUB_DAVG', 'TOT_DEBT_TO_TOT_EQY', 'PROF_MARGIN',
       'OPER_MARGIN', 'WACC', 'PX_LAST', 'ticker'],
      dtype='object')